## Train with imgaug generator

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
import keras
import cv2
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Make {'image_id':depth} dictionary and train-val split

In [38]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()

In [39]:
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values

In [40]:
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)

In [41]:
# make generator
batch_size = 32
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size)

In [42]:
# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation
# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


### Make val data

In [11]:
# X_val = np.zeros((len(val_ids), 128, 128, 3), dtype=np.float32)
# X_feat_val = np.zeros((len(val_ids), 1), dtype=np.float32)
# Y_val_true = np.zeros((len(val_ids), 128, 128, 1), dtype=np.float32)

# for i, im_id in enumerate(val_ids):
# #     image
#     im = cv2.imread('./data/train/images/' + im_id + '.png')
#     im = cv2.resize(im, (128, 128))
#     im = np.array(im, dtype='float32')
#     im /= 255
#     X_val[i] = im
#     #     feature
#     X_feat_val[i] = depth_dict[im_id]
# #     mask
#     mask = cv2.imread('./data/train/masks/' + im_id + '.png')
#     mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
#     mask = np.array((mask > 127.5), dtype='float32')
#     mask = np.expand_dims(mask[:, :, 0], axis=-1)
#     Y_val_true[i] = mask

# val_gen = ([X_val, X_feat_val], Y_val_true)

### Model

In [43]:
unet_m = util.unet_model(min_filter_num=16, with_vec=True)

## Train

In [ ]:

# train_util.train(unet_m, tgs_generator.generate(True), tgs_generator.generate(False), loss=train_util.dice_p_bce, metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef], epochs=3000, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)
train_util.train(unet_m, tgs_generator.generate(True), tgs_generator.generate(False), metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef], epochs=3000, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_block_1 (Conv2D)        (None, 128, 128, 16) 448         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 16) 64          encoder_block_1[0][0]            
__________________________________________________________________________________________________
elu_1 (ELU)                     (None, 128, 128, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

Epoch 1/3000
 - 64s - loss: 0.5011 - dice_coef: 0.2614 - binary_accuracy: 0.7836 - true_positive_rate: 0.6700 - iou_coef: 0.2820 - val_loss: 2.7198 - val_dice_coef: 0.3325 - val_binary_accuracy: 0.7050 - val_true_positive_rate: 0.9489 - val_iou_coef: 0.4317
Finished epoch: 0
{'val_dice_coef': 0.3324884660542011, 'loss': 0.5010661971323269, 'true_positive_rate': 0.6699681810497009, 'iou_coef': 0.28196940555707695, 'val_true_positive_rate': 0.948906180759271, 'val_binary_accuracy': 0.7050313154856364, 'binary_accuracy': 0.7836031176380276, 'val_loss': 2.719836860895157, 'dice_coef': 0.2613912025063308, 'val_iou_coef': 0.4317179936915636}

Epoch 00001: val_loss improved from inf to 2.71984, saving model to ./log/2018_0808_1149/best_weights.hdf5
Epoch 2/3000
 - 57s - loss: 0.3441 - dice_coef: 0.2829 - binary_accuracy: 0.8742 - true_positive_rate: 0.6920 - iou_coef: 0.5326 - val_loss: 1.1465 - val_dice_coef: 0.3505 - val_binary_accuracy: 0.7820 - val_true_positive_rate: 0.9211 - val_iou_coe

Finished epoch: 11
{'val_dice_coef': 0.3911113068461418, 'loss': 0.22419584212229424, 'true_positive_rate': 0.787744222842541, 'iou_coef': 0.6230504798520472, 'val_true_positive_rate': 0.9134279911716779, 'val_binary_accuracy': 0.8648883501688639, 'binary_accuracy': 0.9136207816527062, 'val_loss': 0.4564775340259075, 'lr': 0.001, 'dice_coef': 0.33992304722058403, 'val_iou_coef': 0.6049030274152756}

Epoch 00012: val_loss improved from 0.45877 to 0.45648, saving model to ./log/2018_0808_1149/best_weights.hdf5
Epoch 13/3000
 - 57s - loss: 0.2133 - dice_coef: 0.3452 - binary_accuracy: 0.9205 - true_positive_rate: 0.7936 - iou_coef: 0.6357 - val_loss: 0.8695 - val_dice_coef: 0.3817 - val_binary_accuracy: 0.7609 - val_true_positive_rate: 0.9703 - val_iou_coef: 0.4919
Finished epoch: 12
{'val_dice_coef': 0.3816660866141319, 'loss': 0.21327957119216623, 'true_positive_rate': 0.7935707685873681, 'iou_coef': 0.6356953594487967, 'val_true_positive_rate': 0.9702979127566019, 'val_binary_accuracy'

 - 60s - loss: 0.1888 - dice_coef: 0.3680 - binary_accuracy: 0.9291 - true_positive_rate: 0.8251 - iou_coef: 0.6629 - val_loss: 0.3732 - val_dice_coef: 0.4234 - val_binary_accuracy: 0.9101 - val_true_positive_rate: 0.9390 - val_iou_coef: 0.6523
Finished epoch: 23
{'val_dice_coef': 0.42338312417268753, 'loss': 0.188752469405071, 'true_positive_rate': 0.8251394776953864, 'iou_coef': 0.6628939847356266, 'val_true_positive_rate': 0.9389767150084177, 'val_binary_accuracy': 0.9100974400838217, 'binary_accuracy': 0.9291099862954051, 'val_loss': 0.37319587854047614, 'lr': 0.001, 'dice_coef': 0.36796798810516435, 'val_iou_coef': 0.6522720083594322}

Epoch 00024: val_loss did not improve
Epoch 25/3000
 - 58s - loss: 0.1948 - dice_coef: 0.3656 - binary_accuracy: 0.9240 - true_positive_rate: 0.8150 - iou_coef: 0.6635 - val_loss: 0.3401 - val_dice_coef: 0.4280 - val_binary_accuracy: 0.9022 - val_true_positive_rate: 0.9439 - val_iou_coef: 0.6886
Finished epoch: 24
{'val_dice_coef': 0.427995072056849

 - 63s - loss: 0.1606 - dice_coef: 0.3880 - binary_accuracy: 0.9397 - true_positive_rate: 0.8575 - iou_coef: 0.7009 - val_loss: 0.2706 - val_dice_coef: 0.4370 - val_binary_accuracy: 0.9115 - val_true_positive_rate: 0.9448 - val_iou_coef: 0.6988
Finished epoch: 35
{'val_dice_coef': 0.43699829280376434, 'loss': 0.16063743875813238, 'true_positive_rate': 0.8575077751248154, 'iou_coef': 0.7008844829097236, 'val_true_positive_rate': 0.9447955340147018, 'val_binary_accuracy': 0.9115156332651774, 'binary_accuracy': 0.9397022994523195, 'val_loss': 0.27062104518214863, 'lr': 0.00020000001, 'dice_coef': 0.388006992561301, 'val_iou_coef': 0.6987778221567472}

Epoch 00036: val_loss did not improve
Epoch 37/3000
 - 61s - loss: 0.1573 - dice_coef: 0.3916 - binary_accuracy: 0.9403 - true_positive_rate: 0.8589 - iou_coef: 0.6895 - val_loss: 0.2791 - val_dice_coef: 0.4399 - val_binary_accuracy: 0.8865 - val_true_positive_rate: 0.9482 - val_iou_coef: 0.6759
Finished epoch: 36
{'val_dice_coef': 0.4399184

...

 - 58s - loss: 0.1435 - dice_coef: 0.4021 - binary_accuracy: 0.9460 - true_positive_rate: 0.8693 - iou_coef: 0.7085 - val_loss: 0.1804 - val_dice_coef: 0.4480 - val_binary_accuracy: 0.9275 - val_true_positive_rate: 0.9407 - val_iou_coef: 0.7030
Finished epoch: 106
{'val_dice_coef': 0.4480070744951566, 'loss': 0.1435440281002792, 'true_positive_rate': 0.8692671453829893, 'iou_coef': 0.7085489845767463, 'val_true_positive_rate': 0.9407202030221621, 'val_binary_accuracy': 0.927537759145101, 'binary_accuracy': 0.9460039040477005, 'val_loss': 0.18043309698502222, 'lr': 6.4000005e-08, 'dice_coef': 0.40214044439423946, 'val_iou_coef': 0.7030328611532847}

Epoch 00107: val_loss did not improve
Epoch 108/3000
 - 58s - loss: 0.1363 - dice_coef: 0.4026 - binary_accuracy: 0.9488 - true_positive_rate: 0.8766 - iou_coef: 0.7136 - val_loss: 0.1827 - val_dice_coef: 0.4477 - val_binary_accuracy: 0.9267 - val_true_positive_rate: 0.9413 - val_iou_coef: 0.7019
Finished epoch: 107
{'val_dice_coef': 0.44771

 - 56s - loss: 0.1435 - dice_coef: 0.3984 - binary_accuracy: 0.9459 - true_positive_rate: 0.8709 - iou_coef: 0.7087 - val_loss: 0.1825 - val_dice_coef: 0.4481 - val_binary_accuracy: 0.9267 - val_true_positive_rate: 0.9412 - val_iou_coef: 0.7034
Finished epoch: 118
{'val_dice_coef': 0.44809797902901966, 'loss': 0.14351100870168088, 'true_positive_rate': 0.8709184951388959, 'iou_coef': 0.7087199989053392, 'val_true_positive_rate': 0.9411686261494955, 'val_binary_accuracy': 0.9266792138417562, 'binary_accuracy': 0.9458911541810969, 'val_loss': 0.1825086303676168, 'lr': 1.2800001e-08, 'dice_coef': 0.39844503384275537, 'val_iou_coef': 0.7033729727069536}

Epoch 00119: val_loss did not improve
Epoch 120/3000
 - 63s - loss: 0.1418 - dice_coef: 0.3992 - binary_accuracy: 0.9474 - true_positive_rate: 0.8733 - iou_coef: 0.6975 - val_loss: 0.1831 - val_dice_coef: 0.4483 - val_binary_accuracy: 0.9264 - val_true_positive_rate: 0.9412 - val_iou_coef: 0.7032
Finished epoch: 119
{'val_dice_coef': 0.448

 - 64s - loss: 0.1485 - dice_coef: 0.4013 - binary_accuracy: 0.9451 - true_positive_rate: 0.8757 - iou_coef: 0.6995 - val_loss: 0.1819 - val_dice_coef: 0.4486 - val_binary_accuracy: 0.9269 - val_true_positive_rate: 0.9410 - val_iou_coef: 0.7034
Finished epoch: 130
{'val_dice_coef': 0.44860132535298664, 'loss': 0.14851923094889552, 'true_positive_rate': 0.8756584144130195, 'iou_coef': 0.6995433378465397, 'val_true_positive_rate': 0.9410425126552582, 'val_binary_accuracy': 0.9269397258758545, 'binary_accuracy': 0.9451415661684016, 'val_loss': 0.18189394318809113, 'lr': 2.5600002e-09, 'dice_coef': 0.4013184820561065, 'val_iou_coef': 0.7034089590112368}

Epoch 00131: val_loss did not improve
Epoch 132/3000
 - 60s - loss: 0.1407 - dice_coef: 0.4007 - binary_accuracy: 0.9468 - true_positive_rate: 0.8705 - iou_coef: 0.7045 - val_loss: 0.1809 - val_dice_coef: 0.4485 - val_binary_accuracy: 0.9275 - val_true_positive_rate: 0.9408 - val_iou_coef: 0.7034
Finished epoch: 131
{'val_dice_coef': 0.448

 - 58s - loss: 0.1369 - dice_coef: 0.4016 - binary_accuracy: 0.9493 - true_positive_rate: 0.8756 - iou_coef: 0.7164 - val_loss: 0.1792 - val_dice_coef: 0.4481 - val_binary_accuracy: 0.9275 - val_true_positive_rate: 0.9406 - val_iou_coef: 0.7032
Finished epoch: 142
{'val_dice_coef': 0.44812298690279323, 'loss': 0.13689394018699214, 'true_positive_rate': 0.8755893031346429, 'iou_coef': 0.7164145511450227, 'val_true_positive_rate': 0.9406361455718676, 'val_binary_accuracy': 0.9275472164154053, 'binary_accuracy': 0.9493151595912028, 'val_loss': 0.17923888750374317, 'lr': 5.1200005e-10, 'dice_coef': 0.40163077045347273, 'val_iou_coef': 0.7032249743739764}

Epoch 00143: val_loss did not improve
Epoch 144/3000
 - 58s - loss: 0.1381 - dice_coef: 0.4001 - binary_accuracy: 0.9481 - true_positive_rate: 0.8711 - iou_coef: 0.7039 - val_loss: 0.1815 - val_dice_coef: 0.4477 - val_binary_accuracy: 0.9269 - val_true_positive_rate: 0.9410 - val_iou_coef: 0.7030
Finished epoch: 143
{'val_dice_coef': 0.44

 - 64s - loss: 0.1391 - dice_coef: 0.4015 - binary_accuracy: 0.9485 - true_positive_rate: 0.8721 - iou_coef: 0.7006 - val_loss: 0.1829 - val_dice_coef: 0.4482 - val_binary_accuracy: 0.9266 - val_true_positive_rate: 0.9412 - val_iou_coef: 0.7006
Finished epoch: 154
{'val_dice_coef': 0.44815101101994514, 'loss': 0.1390686854336065, 'true_positive_rate': 0.8721306471480537, 'iou_coef': 0.7006232523426568, 'val_true_positive_rate': 0.9412191212177277, 'val_binary_accuracy': 0.926554282506307, 'binary_accuracy': 0.9484691226605287, 'val_loss': 0.18285453878343105, 'lr': 1.0240001e-10, 'dice_coef': 0.401450280676183, 'val_iou_coef': 0.7006165211399397}

Epoch 00155: val_loss did not improve
Epoch 156/3000
 - 63s - loss: 0.1389 - dice_coef: 0.4011 - binary_accuracy: 0.9468 - true_positive_rate: 0.8775 - iou_coef: 0.7104 - val_loss: 0.1831 - val_dice_coef: 0.4483 - val_binary_accuracy: 0.9264 - val_true_positive_rate: 0.9415 - val_iou_coef: 0.7008
Finished epoch: 155
{'val_dice_coef': 0.448320

## Check iou_coef

In [7]:
from keras.models import load_model
model = load_model('./log/2018_0807_2002/best_weights.hdf5', custom_objects={
    'mean_iou': train_util.mean_iou, 
    'dice_p_bce':train_util.dice_p_bce, 
    'dice_coef':train_util.dice_coef,
    'true_positive_rate':train_util.true_positive_rate,
    'iou_coef':train_util.iou_coef
})

In [29]:
preds = model.predict([X_val, X_feat_val])

In [30]:
preds_t = (preds > 0.5).astype(np.uint8)

In [33]:
ious = []
for i in range(0, len(preds_t)):
    y_true = Y_val_true[i]
    y_pred = preds_t[i]
    ious.append(train_util.compute_iou(y_true, y_pred))

In [36]:
np.mean(ious)

0.7901850010723667

In [37]:
len(train_util.BAD_MASKS)

113

In [41]:
113/(len(preds) * 5)

0.02904884318766067

In [45]:
0.79/1.02

0.7745098039215687

In [52]:
iou = train_util.iou_coef(Y_val_true, np.float32(preds_t))

In [58]:
from keras import backend as K
K.get_value(K.mean((iou)))

0.78863376